In [ ]:
# ==========================================
# 🧱 BLOC 1 — Setup, Helpers & Chargement Données
# ==========================================

from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import statsmodels.api as sm
from itertools import product
import plotly.express as px
import plotly.graph_objects as go

# Configuration de base
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 180)
np.random.seed(42)

# Réglages graphiques
px.defaults.template = "plotly_white"
px.defaults.width = 1000
px.defaults.height = 520

# ==========================================
# 📁 Chemins & Fichiers
# ==========================================
DATA = Path("../data_clean")
FILES = {
    "common_FR_long": DATA / "ODISSEE/common_FR_long.csv",
    "vacsi_fr_extended": DATA / "VACSI/vacsi_fr_extended.csv",
    "google_mobility_fr_weekly": DATA / "GOOGLE/google_mobility_fr_weekly.csv",
    "coviprev_reg_weekly": DATA / "COVIPREV/coviprev_reg_weekly.csv",
    "meteo_fr_weekly": DATA / "METEO/meteo_fr_weekly.csv",
    "erviss_fr_weekly": DATA / "ERVISS/erviss_fr_weekly.csv",
}

missing = [k for k, p in FILES.items() if not p.exists()]
if missing:
    raise FileNotFoundError(f"❌ Fichiers manquants: {missing}")
print("✅ Tous les fichiers nécessaires sont disponibles.")

COVID_START = pd.Timestamp("2020-03-01")
VACC_START  = pd.Timestamp("2021-01-01")

LAG_VACC, LAG_MNP, LAG_WORK = 4, 8, 9
SEASON_PERIOD = 52
print(f"⏱️ COVID_START={COVID_START.date()}, VACC_START={VACC_START.date()} | Lags: {LAG_VACC, LAG_MNP, LAG_WORK}")

# ==========================================
# 🧩 Fonctions utilitaires
# ==========================================

def keyify(df: pd.DataFrame) -> pd.DataFrame:
    """Ajoute les colonnes ISO-year et ISO-week à partir d'une colonne date_monday"""
    iso = pd.to_datetime(df["date_monday"]).dt.isocalendar()
    df["year_iso"] = iso["year"].astype(int)
    df["week_iso_num"] = iso["week"].astype(int)
    return df

def zscore(s):
    """Normalisation standardisée"""
    return (s - s.mean()) / s.std(ddof=0) if s.std(ddof=0) != 0 else s * 0

def build_time_features(df, period=52):
    """Ajoute les composantes saisonnières sin/cos"""
    df = df.copy()
    df["t"] = np.arange(len(df))
    df["sin52"] = np.sin(2 * np.pi * df["t"] / period)
    df["cos52"] = np.cos(2 * np.pi * df["t"] / period)
    return df

def load_datasets(files):
    """Charge l'ensemble des fichiers CSV listés"""
    data = {}
    for name, path in files.items():
        data[name] = pd.read_csv(path)
        print(f"✅ {name} chargé ({data[name].shape[0]} lignes)")
    return data

def merge_exog(rsv_df, vac_df, work_df, cov_df):
    """Fusionne RSV avec les variables exogènes (vaccination, mobilité, CoviPrev)"""
    merged = (
        rsv_df[["date_monday", "year_iso", "week_iso_num"]]
        .merge(vac_df, on=["year_iso", "week_iso_num"], how="left")
        .merge(work_df, on=["year_iso", "week_iso_num"], how="left")
        .merge(cov_df, on=["year_iso", "week_iso_num"], how="left")
        .set_index("date_monday")
        .sort_index()
    )
    return merged

def build_model_matrix(df, lags=(4, 8, 9), mask_vars=None):
    """Construit la matrice des variables explicatives, avec décalages (lags)"""
    lag_vac, lag_mnp, lag_work = lags
    df["work_red"] = zscore(-df["work"])  # inversion pour cohérence (baisse = confinement)
    if mask_vars:
        for v in mask_vars:
            df[v] = zscore(df[v])
        df["MNP_score"] = df[mask_vars + ["work_red"]].mean(axis=1)
    else:
        df["MNP_score"] = zscore(df["work_red"])

    X = pd.DataFrame(index=df.index)
    X["cov12_lag"] = df["couv_complet"].shift(lag_vac)
    X["MNP_lag"]   = df["MNP_score"].shift(lag_mnp)
    X["work_lag"]  = df["work"].shift(lag_work)
    return build_time_features(X)

# --- Extension CoviPrev ---
def extend_coviprev(df, mode="constant", weeks=20):
    """
    Étend la série CoviPrev même sans colonne 'date_monday'.
    - Si index = (year_iso, week_iso_num) → crée un index temporel synthétique
    - mode: 'constant', 'trend', ou 'smooth'
    """
    df_ext = df.copy()
    
    # Cas 1 : pas de colonne date_monday, mais index multi-niveau (année/semaine)
    if "date_monday" not in df_ext.columns:
        if isinstance(df_ext.index, pd.MultiIndex):
            tmp = df_ext.reset_index()
            # Reconstruire la date du lundi de chaque semaine ISO
            tmp["date_monday"] = tmp.apply(
                lambda r: pd.to_datetime(f"{int(r['year_iso'])}-W{int(r['week_iso_num']):02d}-1", format="%G-W%V-%u"),
                axis=1
            )
            tmp = tmp.sort_values("date_monday")
            df_ext = tmp.set_index("date_monday")
        else:
            raise ValueError("⚠️ L'index CoviPrev ne contient pas les colonnes nécessaires (year_iso, week_iso_num).")

    # Étendre avec les dernières valeurs
    last_date = df_ext.index.max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(weeks=1), periods=weeks, freq="W-MON")

    if mode == "constant":
        last_row = df_ext.iloc[-1]
        ext = pd.DataFrame([last_row.values] * len(future_dates), columns=df_ext.columns, index=future_dates)

    elif mode == "trend":
        ext = pd.DataFrame(index=future_dates, columns=df_ext.columns)
        for col in df_ext.columns:
            y = df_ext[col].dropna().values
            if len(y) > 2:
                coef = np.polyfit(np.arange(len(y)), y, 1)
                trend = np.polyval(coef, np.arange(len(y), len(y) + weeks))
                ext[col] = trend
            else:
                ext[col] = df_ext[col].iloc[-1]
    elif mode == "smooth":
        ext = pd.DataFrame(index=future_dates, columns=df_ext.columns)
        for col in df_ext.columns:
            ext[col] = df_ext[col].tail(4).mean()
    else:
        raise ValueError("Mode non reconnu: 'constant', 'trend' ou 'smooth'")

    df_out = pd.concat([df_ext, ext])
    return df_out

# ==========================================
# 📊 Chargement des données
# ==========================================
data = load_datasets(FILES)

# === RSV principal ===
common = keyify(data["common_FR_long"])
mask = (common["topic"] == "RSV") & (common["geo_level"] == "FR")
age_used = next(a for a in ["00-04 ans", "0-1 an", "Tous âges"] if ((mask) & (common["classe_d_age"] == a)).any())
mask &= (common["classe_d_age"] == age_used)

ycol = "taux_passages_urgences" if "taux_passages_urgences" in common.columns else "taux_sos"
rsv = common.loc[mask, ["date_monday", "year_iso", "week_iso_num", ycol]].rename(columns={ycol: "RSV"})
rsv["date_monday"] = pd.to_datetime(rsv["date_monday"])
rsv = rsv.sort_values("date_monday")
print(f"✅ RSV prêt ({age_used}) — {rsv.shape[0]} lignes")

# === VACSI ===
vac = keyify(data["vacsi_fr_extended"]).query("geo_level=='FR'")[["year_iso","week_iso_num","couv_complet"]]

# === GOOGLE MOBILITY ===
gm  = keyify(data["google_mobility_fr_weekly"])
work = gm.query("geo_level=='FR' & indicator=='workplaces'")[["year_iso","week_iso_num","value"]].rename(columns={"value": "work"})

# === COVIPREV ===
cov = keyify(data["coviprev_reg_weekly"])
mask_vars = ["port_du_masque","lavage_des_mains","aeration_du_logement","saluer_sans_serrer_la_main"]
cov_nat = cov[cov["indicator"].isin(mask_vars)].groupby(["year_iso","week_iso_num","indicator"])["value"].mean().unstack()

# === 🔧 Étendre CoviPrev pour les semaines futures ===
cov_nat = extend_coviprev(cov_nat, mode="constant", weeks=20)
print(f"✅ CoviPrev agrégé et étendu ({len(cov_nat)} semaines)")

# === Fusion ===
X_base = merge_exog(rsv, vac, work, cov_nat)
X_full = build_model_matrix(X_base, lags=(LAG_VACC, LAG_MNP, LAG_WORK), mask_vars=mask_vars)

df_base = rsv.set_index("date_monday")[["RSV"]].join(X_full, how="left").dropna().sort_index()
print(f"✅ Base finale prête : {df_base.shape}")

# ==========================================
# 🧱 BLOC 2 — OLS optimisé (préparation)
# ==========================================
best_r2, best_lags = -np.inf, (LAG_VACC, LAG_MNP, LAG_WORK)
for lv, lm, lw in product(range(2,9), range(4,13), range(4,13)):
    X_tmp = build_model_matrix(X_base, lags=(lv,lm,lw), mask_vars=mask_vars)
    df_tmp = rsv.set_index("date_monday")[["RSV"]].join(X_tmp).dropna()
    if len(df_tmp) < 40: continue
    m = sm.OLS(df_tmp["RSV"], sm.add_constant(df_tmp[["cov12_lag","MNP_lag","work_lag","sin52","cos52"]])).fit()
    if m.rsquared_adj > best_r2:
        best_r2, best_lags = m.rsquared_adj, (lv,lm,lw)
print(f"🥇 Lags optimaux : {best_lags}")

# === Construction de df_opt ===
X_full_opt = build_model_matrix(X_base, lags=best_lags, mask_vars=mask_vars)
df_opt = rsv.set_index("date_monday")[["RSV"]].join(X_full_opt).dropna()

# Ajout météo et variables dérivées
meteo = keyify(data["meteo_fr_weekly"])[["year_iso","week_iso_num","tmean"]]
df_opt = keyify(df_opt.reset_index()).merge(meteo,on=["year_iso","week_iso_num"],how="left").set_index("date_monday").sort_index()
df_opt["tmean_z"]    = zscore(df_opt["tmean"])
df_opt["vacc_x_mnp"] = df_opt["cov12_lag"] * df_opt["MNP_lag"]
df_opt["RSV_lag1"]   = df_opt["RSV"].shift(1)
df_opt["RSV_lag2"]   = df_opt["RSV"].shift(2)
df_opt = df_opt.dropna()

print(f"✅ Base optimisée prête : {df_opt.shape}")

In [ ]:
# ==========================================
# 🧱 BLOC 2 — OLS (base + optimisé + causal)
# ==========================================
import statsmodels.api as sm
import plotly.graph_objects as go

print("🚀 Lancement du bloc OLS...")

# ==========================================
# 1️⃣ OLS BASE
# ==========================================
Y = df_base["RSV"].astype(float)
X_cols_base = ["cov12_lag", "MNP_lag", "work_lag", "sin52", "cos52"]
Xb = df_base[X_cols_base]

ols_base = sm.OLS(Y, sm.add_constant(Xb)).fit(cov_type="HC3")

print(f"✅ OLS base entraîné : R²_adj={ols_base.rsquared_adj:.3f}, AIC={ols_base.aic:.1f}")

# --- Graphique ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_base.index, y=df_base["RSV"], name="RSV observé", mode="lines", line=dict(color="black")))
fig.add_trace(go.Scatter(x=df_base.index, y=ols_base.fittedvalues, name="OLS fitted", mode="lines", line=dict(color="blue", dash="dot")))
fig.add_vline(x=COVID_START, line_dash="dash", line_color="red")
fig.add_vline(x=VACC_START, line_dash="dash", line_color="green")
fig.update_layout(title="RSV — OLS base (série complète)", xaxis_title="Semaine", yaxis_title="RSV")
fig.show()


# ==========================================
# 2️⃣ OLS OPTIMISÉ (avec variables enrichies)
# ==========================================
Xo = df_opt[[
    "cov12_lag","MNP_lag","work_lag","tmean_z",
    "vacc_x_mnp","RSV_lag1","RSV_lag2","sin52","cos52"
]]
Y_opt = df_opt["RSV"].astype(float)

ols_opt = sm.OLS(Y_opt, sm.add_constant(Xo)).fit(cov_type="HC3")

print(f"✅ OLS optimisé entraîné : R²_adj={ols_opt.rsquared_adj:.3f}, AIC={ols_opt.aic:.1f}, DW={sm.stats.stattools.durbin_watson(ols_opt.resid):.2f}")

# --- Graphique comparatif ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_opt.index, y=df_opt["RSV"], name="RSV observé", mode="lines", line=dict(color="black")))
fig.add_trace(go.Scatter(x=df_opt.index, y=ols_opt.fittedvalues, name="OLS optimisé", mode="lines", line=dict(color="royalblue", dash="dot")))
fig.add_vline(x=COVID_START, line_dash="dash", line_color="red")
fig.add_vline(x=VACC_START, line_dash="dash", line_color="green")
fig.update_layout(title="RSV — OLS optimisé (série complète)", xaxis_title="Semaine", yaxis_title="RSV")
fig.show()


# ==========================================
# 3️⃣ OLS CAUSAL (décalage MNP = +3 semaines)
# ==========================================
LAG_MNP_EFFECT = 3  # effet retardé estimé à 3 semaines
df_opt["MNP_lag_causal"] = df_opt["MNP_lag"].shift(LAG_MNP_EFFECT)
df_opt["vacc_x_mnp_causal"] = df_opt["cov12_lag"] * df_opt["MNP_lag_causal"]

# Préparation des variables
Xo_causal = df_opt[[
    "cov12_lag", "MNP_lag_causal", "work_lag", "tmean_z",
    "vacc_x_mnp_causal", "RSV_lag1", "RSV_lag2", "sin52", "cos52"
]]

# Réalignement propre : garder uniquement les lignes valides pour X ET Y
valid_idx = Xo_causal.dropna().index.intersection(df_opt.index)
Xo_causal = Xo_causal.loc[valid_idx].dropna()
Y_causal = df_opt.loc[valid_idx, "RSV"]

# Vérification
print(f"Alignement OK : {len(Xo_causal)} lignes valides sur {len(df_opt)}")

# Fit du modèle
ols_causal = sm.OLS(Y_causal, sm.add_constant(Xo_causal)).fit(cov_type="HC3")

print(f"✅ OLS causal recalibré : R²_adj={ols_causal.rsquared_adj:.3f}, AIC={ols_causal.aic:.1f}, DW={sm.stats.stattools.durbin_watson(ols_causal.resid):.2f}")
display(ols_causal.summary())



# ==========================================
# 4️⃣ COMPARAISON VISUELLE — BASE vs OPTIMISÉ
# ==========================================
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_base.index, y=df_base["RSV"], name="RSV observé", line=dict(color="black", width=2)))
fig.add_trace(go.Scatter(x=df_base.index, y=ols_base.fittedvalues, name="OLS base", line=dict(color="orange", dash="dot")))
fig.add_trace(go.Scatter(x=df_opt.index, y=ols_opt.fittedvalues, name="OLS optimisé", line=dict(color="royalblue", dash="dot")))
fig.add_vline(x=COVID_START, line_dash="dash", line_color="red")
fig.add_vline(x=VACC_START, line_dash="dash", line_color="green")
fig.update_layout(title="Comparaison OLS base vs OLS optimisé", xaxis_title="Semaine", yaxis_title="RSV (taux hebdo)")
fig.show()


# ==========================================
# 5️⃣ INTERPRÉTATION RAPIDE
# ==========================================
print("\n📊 Résumé comparatif des performances :")
print(f"  OLS base      → R²_adj = {ols_base.rsquared_adj:.3f}, DW = {sm.stats.stattools.durbin_watson(ols_base.resid):.2f}")
print(f"  OLS optimisé  → R²_adj = {ols_opt.rsquared_adj:.3f}, DW = {sm.stats.stattools.durbin_watson(ols_opt.resid):.2f}")
print(f"  OLS causal    → R²_adj = {ols_causal.rsquared_adj:.3f}, DW = {sm.stats.stattools.durbin_watson(ols_causal.resid):.2f}")

# Interprétation qualitative
from IPython.display import Markdown, display
display(Markdown(f"""
### 🧩 Lecture rapide des résultats OLS

- **OLS base** : premier ajustement global du RSV aux variables comportementales et saisonnières.  
  → R² ajusté ≈ {ols_base.rsquared_adj:.2f} → modèle explicatif modéré, saisonnalité captée mais sans finesse.  
  → DW ≈ {sm.stats.stattools.durbin_watson(ols_base.resid):.2f} → résidus autocorrélés (modèle encore rigide).

- **OLS optimisé** : ajout des décalages optimaux, météo et effets d’inertie (RSV lag1/lag2).  
  → R² ajusté ≈ {ols_opt.rsquared_adj:.2f} → excellent pouvoir explicatif.  
  → DW ≈ {sm.stats.stattools.durbin_watson(ols_opt.resid):.2f} → erreurs indépendantes → modèle robuste.

- **OLS causal** : introduit un décalage de 3 semaines pour capturer les effets différés des gestes barrières.  
  → R² ajusté ≈ {ols_causal.rsquared_adj:.2f}, cohérent avec le modèle optimisé.  
  → L’interprétation devient plus réaliste pour la lecture causale des scénarios.
"""))


🚀 Lancement du bloc OLS...
✅ OLS base entraîné : R²_adj=0.496, AIC=1759.1


✅ OLS optimisé entraîné : R²_adj=0.966, AIC=1315.4, DW=1.99


Alignement OK : 108 lignes valides sur 111
✅ OLS causal recalibré : R²_adj=0.968, AIC=1276.4, DW=2.01


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RSV   R-squared:                       0.970
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     200.3
Date:                Thu, 06 Nov 2025   Prob (F-statistic):           5.53e-59
Time:                        10:38:57   Log-Likelihood:                -628.20
No. Observations:                 108   AIC:                             1276.
Df Residuals:                      98   BIC:                             1303.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                92.9533     60.669      1.532      0.125     -25.956     211.863
cov12_lag             0.4811      3.219      0.149      0.881      -5.828       6.790
MNP_lag_causal      -30.6952     32.166     -0.954      0.340     -93.739      32.349
work_lag              0.2258      1.020      0.221      0.825      -1.774       2.226
tmean_z             -13.3189     14.028     -0.949      0.342     -40.812      14.175
vacc_x_mnp_causal     2.4524      2.016      1.217      0.224      -1.498       6.403
RSV_lag1              1.6282      0.107     15.158      0.000       1.418       1.839
RSV_lag2             -0.7504      0.112     -6.683      0.000      -0.971      -0.530
sin52                50.8073     19.378      2.622      0.009      12.827      88.787
cos52                12.5414     15.065      0.833      0.405     -16.985      42.068
==============================================================================
Omnibus:                       15.448   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.148
Skew:                           0.430   Prob(JB):                     3.16e-09
Kurtosis:                       5.821   Cond. No.                     7.47e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 7.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""


📊 Résumé comparatif des performances :
  OLS base      → R²_adj = 0.496, DW = 0.11
  OLS optimisé  → R²_adj = 0.966, DW = 1.99
  OLS causal    → R²_adj = 0.968, DW = 2.01



### 🧩 Lecture rapide des résultats OLS

- **OLS base** : premier ajustement global du RSV aux variables comportementales et saisonnières.  
  → R² ajusté ≈ 0.50 → modèle explicatif modéré, saisonnalité captée mais sans finesse.  
  → DW ≈ 0.11 → résidus autocorrélés (modèle encore rigide).

- **OLS optimisé** : ajout des décalages optimaux, météo et effets d’inertie (RSV lag1/lag2).  
  → R² ajusté ≈ 0.97 → excellent pouvoir explicatif.  
  → DW ≈ 1.99 → erreurs indépendantes → modèle robuste.

- **OLS causal** : introduit un décalage de 3 semaines pour capturer les effets différés des gestes barrières.  
  → R² ajusté ≈ 0.97, cohérent avec le modèle optimisé.  
  → L’interprétation devient plus réaliste pour la lecture causale des scénarios.


In [ ]:
# ==========================================
# 🔁 BLOC 3 — ITS (rebuild) sur ODISSEE pur
# ==========================================
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
import plotly.graph_objects as go

print("🚀 ITS reconstruit sur ODISSEE observé (2019–…)")

# 0) Source = ODISSEE observé uniquement (pas d'exogènes)
rsv_obs = rsv.copy()  # rsv vient de ton BLOC 1 (ODISSEE)
rsv_obs = rsv_obs.sort_values("date_monday").set_index("date_monday")[["RSV"]]

# Assurer une fréquence hebdo (lundi ISO) continue
idx_full = pd.date_range(start=rsv_obs.index.min(), end=rsv_obs.index.max(), freq="W-MON")
rsv_obs = rsv_obs.reindex(idx_full)
# (option) si trous très ponctuels, tu peux interpoler légèrement :
# rsv_obs["RSV"] = rsv_obs["RSV"].interpolate(limit_direction="both")

# 1) ITS BASE — ruptures COVID / Vaccin
df_its = rsv_obs.copy()
df_its["t"] = np.arange(len(df_its))
df_its["post_covid"] = (df_its.index >= COVID_START).astype(int)
df_its["post_vacc"]  = (df_its.index >= VACC_START).astype(int)
df_its["t_post_covid"] = df_its["t"] * df_its["post_covid"]
df_its["t_post_vacc"]  = df_its["t"] * df_its["post_vacc"]
# Saison simple
df_its["sin52"] = np.sin(2*np.pi*df_its["t"]/52)
df_its["cos52"] = np.cos(2*np.pi*df_its["t"]/52)

Y = df_its["RSV"].astype(float)
X = df_its[["t","sin52","cos52","post_covid","t_post_covid","post_vacc","t_post_vacc"]]

its = sm.OLS(Y, sm.add_constant(X)).fit(cov_type="HAC", cov_kwds={"maxlags":12})
print(f"✅ ITS base (ODISSEE pur) : AIC={its.aic:.1f}, BIC={its.bic:.1f}, DW={sm.stats.stattools.durbin_watson(its.resid):.2f}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_its.index, y=df_its["RSV"], name="RSV observé", mode="lines", line=dict(color="black", width=2)))
fig.add_trace(go.Scatter(x=df_its.index, y=its.fittedvalues, name="ITS fitted", mode="lines", line=dict(color="royalblue", dash="dot", width=2)))
# Lignes verticales robustes (add_shape)
fig.add_shape(type="line", x0=COVID_START, x1=COVID_START, y0=0, y1=1, xref="x", yref="paper", line=dict(color="red", width=2, dash="dash"))
fig.add_annotation(x=COVID_START, y=1, text="COVID", showarrow=False, yref="paper", font=dict(color="red", size=10))
fig.add_shape(type="line", x0=VACC_START, x1=VACC_START, y0=0, y1=1, xref="x", yref="paper", line=dict(color="green", width=2, dash="dash"))
fig.add_annotation(x=VACC_START, y=1, text="Vaccin", showarrow=False, yref="paper", font=dict(color="green", size=10))
fig.update_layout(title="ITS (base) — ODISSEE observé (2019–…)", xaxis_title="Semaine", yaxis_title="RSV (taux)")
fig.show()

# 2) ITS OPTIMISÉ — dates ±28j & Fourier (toujours sur ODISSEE pur)
def add_fourier_indexed(df, K=1, period=52):
    out = df.copy()
    t = np.arange(len(out))
    for k in range(1, K+1):
        out[f"sin{k}"] = np.sin(2*np.pi*k*t/period)
        out[f"cos{k}"] = np.cos(2*np.pi*k*t/period)
    return out

def make_its_design_obs(series, covid_date, vacc_date, K=1):
    dfX = series.copy()
    dfX["t"] = np.arange(len(dfX))
    dfX["post_covid"] = (dfX.index >= covid_date).astype(int)
    dfX["post_vacc"]  = (dfX.index >= vacc_date).astype(int)
    dfX["t_post_covid"] = dfX["t"] * dfX["post_covid"]
    dfX["t_post_vacc"]  = dfX["t"] * dfX["post_vacc"]
    dfX = add_fourier_indexed(dfX, K=K)
    y = dfX["RSV"].astype(float)
    Xcols = ["t","post_covid","t_post_covid","post_vacc","t_post_vacc"] + \
            [f"sin{k}" for k in range(1,K+1)] + [f"cos{k}" for k in range(1,K+1)]
    X = dfX[Xcols]
    hac_lags = int(np.clip(np.sqrt(len(dfX)),8,24))
    fit = sm.OLS(y, sm.add_constant(X)).fit(cov_type="HAC", cov_kwds={"maxlags": hac_lags})
    return fit, dfX

steps = np.array([-28,-14,0,14,28], dtype="timedelta64[D]")
candidates_covid = [COVID_START + pd.to_timedelta(int(s.astype(int)), unit="D") for s in steps]
candidates_vacc  = [VACC_START  + pd.to_timedelta(int(s.astype(int)), unit="D") for s in steps]
Ks = [1,2,3]

best = {"aic": np.inf}
for K in Ks:
    for cdate in candidates_covid:
        for vdate in candidates_vacc:
            if vdate <= cdate: 
                continue
            try:
                fit, dfX = make_its_design_obs(rsv_obs.copy(), cdate, vdate, K=K)
                if fit.aic < best["aic"]:
                    best = {"aic": fit.aic, "K": K, "covid": cdate, "vacc": vdate, "fit": fit, "df": dfX}
            except Exception:
                pass

its_best = best["fit"]; df_plot = best["df"]
print(f"🥇 ITS optimisé (ODISSEE pur) : AIC={best['aic']:.1f} | K={best['K']} | COVID={best['covid'].date()} | VACC={best['vacc'].date()}")

# Diagnostics
dw = sm.stats.stattools.durbin_watson(its_best.resid)
lb = acorr_ljungbox(its_best.resid, lags=[8,12,24], return_df=True)[["lb_stat","lb_pvalue"]]
print(f"Durbin–Watson={dw:.3f}")
display(lb)

# Plot final (robuste)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot["RSV"], mode="lines", name="RSV observé", line=dict(color="black", width=2)))
fig.add_trace(go.Scatter(x=df_plot.index, y=its_best.fittedvalues, mode="lines", name="ITS optimisé", line=dict(color="royalblue", dash="dot", width=3)))
fig.add_shape(type="line", x0=best["covid"], x1=best["covid"], y0=0, y1=1, xref="x", yref="paper", line=dict(color="red", width=2, dash="dash"))
fig.add_annotation(x=best["covid"], y=1, text="COVID", showarrow=False, yref="paper", font=dict(color="red", size=10))
fig.add_shape(type="line", x0=best["vacc"], x1=best["vacc"], y0=0, y1=1, xref="x", yref="paper", line=dict(color="green", width=2, dash="dash"))
fig.add_annotation(x=best["vacc"], y=1, text="Vaccin", showarrow=False, yref="paper", font=dict(color="green", size=10))
fig.update_layout(title=f"ITS optimisé — ODISSEE observé (K={best['K']})", xaxis_title="Semaine", yaxis_title="RSV (taux)")
fig.show()

🚀 ITS reconstruit sur ODISSEE observé (2019–…)
✅ ITS base (ODISSEE pur) : AIC=4276.3, BIC=4305.7, DW=0.14


🥇 ITS optimisé (ODISSEE pur) : AIC=4079.2 | K=3 | COVID=2020-03-29 | VACC=2021-01-15
Durbin–Watson=0.209


,lb_stat,lb_pvalue
8,631.083238,4.843623e-131
12,632.113624,1.461565e-127
24,728.503888,2.482761e-138


In [ ]:
import pandas as pd
b = its_best.params; se = its_best.bse; p = its_best.pvalues
out = pd.DataFrame({
    "term": ["level_pre","slope_pre","jump_covid","Δslope_post_covid","jump_vacc","Δslope_post_vacc"],
    "estimate": [b["const"], b["t"], b["post_covid"], b["t_post_covid"], b["post_vacc"], b["t_post_vacc"]],
    "se": [se.get(k, float('nan')) for k in ["const","t","post_covid","t_post_covid","post_vacc","t_post_vacc"]],
    "pval": [p.get(k, float('nan')) for k in ["const","t","post_covid","t_post_covid","post_vacc","t_post_vacc"]],
})
# Pentes par phase, pour lecture directe
slopes = pd.Series({
    "slope_pre": b["t"],
    "slope_post_covid": b["t"] + b["t_post_covid"],
    "slope_post_vacc":  b["t"] + b["t_post_covid"] + b["t_post_vacc"],
})

In [ ]:
print(out)

NameError: name 'out' is not defined

In [ ]:
dfsdf

In [ ]:
# ==========================================
# 🧱 BLOC 4 — SARIMAX (ODISSEE complet 2019–2025)
# ==========================================
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
import plotly.graph_objects as go

print("🚀 Lancement du modèle SARIMAX sur la série complète ODISSEE (2019–2025)")

# --- 1️⃣ Base RSV observée (comme dans ITS)
df_sx = rsv_obs.copy()  # RSV complet (ODISSEE)
df_sx.index = pd.to_datetime(df_sx.index)
df_sx = df_sx.sort_index()
df_sx["t"] = np.arange(len(df_sx))

# --- 2️⃣ Variables indicatrices de rupture
df_sx["post_covid"] = (df_sx.index >= COVID_START).astype(int)
df_sx["post_vacc"]  = (df_sx.index >= VACC_START).astype(int)
df_sx["t_post_covid"] = df_sx["t"] * df_sx["post_covid"]
df_sx["t_post_vacc"]  = df_sx["t"] * df_sx["post_vacc"]

# --- 3️⃣ Composantes saisonnières (sin/cos annuels)
df_sx["sin52"] = np.sin(2 * np.pi * df_sx["t"] / 52)
df_sx["cos52"] = np.cos(2 * np.pi * df_sx["t"] / 52)

# --- 4️⃣ Exogènes internes (structurelle uniquement)
exog_cols = ["t", "post_covid", "post_vacc", "t_post_covid", "t_post_vacc", "sin52", "cos52"]
exog = df_sx[exog_cols]
y = df_sx["RSV"].astype(float).fillna(method="ffill")

print(f"✅ Données prêtes : y={len(y)} semaines, X={len(exog.columns)} exogènes")

# --- 5️⃣ Recherche simple de paramètres
pdq_list  = [(p,d,q) for p in [0,1,2] for d in [0,1] for q in [0,1,2]]
PDQ_list  = [(P,1,Q,52) for P in [0,1] for Q in [0,1]]

best = {"aic": np.inf}
for order in pdq_list:
    for seas in PDQ_list:
        try:
            mod = SARIMAX(y, exog=exog, order=order, seasonal_order=seas,
                          enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
            if mod.aic < best["aic"]:
                best = {"model": mod, "aic": mod.aic, "bic": mod.bic,
                        "order": order, "seasonal": seas}
        except:
            pass

sarimax_best = best["model"]
print(f"🏆 SARIMAX optimisé: order={best['order']}×{best['seasonal']} | AIC={best['aic']:.1f}")

# --- 6️⃣ Diagnostic rapide
resid = sarimax_best.resid
dw = sm.stats.stattools.durbin_watson(resid)
lb = acorr_ljungbox(resid, lags=[8,12,24], return_df=True)[["lb_stat","lb_pvalue"]]
print(f"Durbin–Watson={dw:.3f}")
display(lb)

# --- 7️⃣ Courbe ajustée
y_fit = sarimax_best.fittedvalues
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_sx.index, y=y, name="RSV observé (ODISSEE complet)", line=dict(color="black", width=2)))
fig.add_trace(go.Scatter(x=y_fit.index, y=y_fit, name="SARIMAX fitted", line=dict(color="royalblue", dash="dot")))
fig.add_shape(type="line", x0=COVID_START, x1=COVID_START, y0=0, y1=1, xref="x", yref="paper",
              line=dict(color="red", width=2, dash="dash"))
fig.add_annotation(x=COVID_START, y=1, text="COVID", showarrow=False, yref="paper", font=dict(color="red", size=10))
fig.add_shape(type="line", x0=VACC_START, x1=VACC_START, y0=0, y1=1, xref="x", yref="paper",
              line=dict(color="green", width=2, dash="dash"))
fig.add_annotation(x=VACC_START, y=1, text="Vaccin", showarrow=False, yref="paper", font=dict(color="green", size=10))
fig.update_layout(title="SARIMAX — ODISSEE observé (2019–2025)",
                  xaxis_title="Semaine ISO", yaxis_title="RSV (taux hebdo)", height=520)
fig.show()

# --- 8️⃣ Pseudo-R²
ss_res = ((y - y_fit)**2).sum()
ss_tot = ((y - y.mean())**2).sum()
pseudo_r2 = 1 - ss_res/ss_tot
print(f"Pseudo-R² ≈ {pseudo_r2:.3f}")


🚀 Lancement du modèle SARIMAX sur la série complète ODISSEE (2019–2025)
✅ Données prêtes : y=293 semaines, X=7 exogènes


In [ ]:
# ==========================================
# 🧩 BLOC — Scénarios contrefactuels OLS vs SARIMAX (corrigé v2)
# ==========================================
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm

print("🚀 Simulation des scénarios contrefactuels — OLS vs SARIMAX")

# --- 1️⃣ Base commune : on part de df_opt (plus complet que df_sx)
base_df = df_opt.copy()
base_df = base_df.sort_index()

# --- 2️⃣ On vérifie / reconstruit toutes les colonnes nécessaires
required_cols = [
    "cov12_lag", "MNP_lag", "work_lag", "tmean_z", "vacc_x_mnp",
    "RSV_lag1", "RSV_lag2", "sin52", "cos52"
]
for col in required_cols:
    if col not in base_df.columns:
        base_df[col] = 0

# --- 3️⃣ Colonnes exactes du modèle OLS
ols_features = [c for c in ols_opt.model.exog_names if c != "const"]

# --- 4️⃣ Fonction de simulation alignée
def simulate_scenario(df, model, model_type="OLS", vacc_factor=1.0, mnp_factor=1.0, label=""):
    sim = df.copy()
    sim["cov12_lag"] *= vacc_factor
    sim["MNP_lag"] *= mnp_factor
    sim["vacc_x_mnp"] = sim["cov12_lag"] * sim["MNP_lag"]

    if model_type == "OLS":
        # Variables alignées sur le modèle
        X_sim = sm.add_constant(sim[ols_features], has_constant='add')
        sim["pred"] = model.predict(X_sim)
    else:  # SARIMAX
        exog_cols = ["t", "post_covid", "post_vacc", "t_post_covid", "t_post_vacc", "sin52", "cos52"]
        for col in exog_cols:
            if col not in sim.columns:
                sim[col] = np.linspace(0, 1, len(sim))  # fallback neutre
        sim["pred"] = model.get_prediction(exog=sim[exog_cols]).predicted_mean

    sim["Scenario"] = label
    return sim

# --- 5️⃣ Scénarios principaux
scenarios_ols = pd.concat([
    simulate_scenario(base_df, ols_opt, "OLS", 1.0, 1.0, "Observé (OLS)"),
    simulate_scenario(base_df, ols_opt, "OLS", 0.0, 1.0, "Sans vaccination (OLS)"),
    simulate_scenario(base_df, ols_opt, "OLS", 1.0, 0.0, "Sans MNP (OLS)"),
    simulate_scenario(base_df, ols_opt, "OLS", 1.0, 1.5, "MNP maintenus +50% (OLS)")
])

scenarios_sx = pd.concat([
    simulate_scenario(base_df, sarimax_best, "SARIMAX", 1.0, 1.0, "Observé (SARIMAX)"),
    simulate_scenario(base_df, sarimax_best, "SARIMAX", 0.0, 1.0, "Sans vaccination (SARIMAX)"),
    simulate_scenario(base_df, sarimax_best, "SARIMAX", 1.0, 0.0, "Sans MNP (SARIMAX)"),
    simulate_scenario(base_df, sarimax_best, "SARIMAX", 1.0, 1.5, "MNP maintenus +50% (SARIMAX)")
])

# --- 6️⃣ Résumés des deltas cumulés
def delta_summary(df_all, obs_label):
    base = df_all[df_all["Scenario"] == obs_label]["pred"].values
    summary = []
    for scen in df_all["Scenario"].unique():
        if scen == obs_label:
            continue
        sub = df_all[df_all["Scenario"] == scen]["pred"].values
        delta = 100 * (sub.sum() - base.sum()) / base.sum()
        summary.append({"Scénario": scen, "Δ_cumulé_%": round(delta, 2)})
    return pd.DataFrame(summary)

summary_ols = delta_summary(scenarios_ols, "Observé (OLS)")
summary_sx  = delta_summary(scenarios_sx, "Observé (SARIMAX)")

print("📊 Effets cumulés OLS :")
display(summary_ols)
print("📊 Effets cumulés SARIMAX :")
display(summary_sx)

# --- 7️⃣ Visualisation comparative
fig = go.Figure()

for scen, color in zip(
    ["Observé (OLS)", "Sans vaccination (OLS)", "Sans MNP (OLS)", "MNP maintenus +50% (OLS)"],
    ["black", "orange", "red", "green"]
):
    dash = "solid" if scen == "Observé (OLS)" else "dot"
    fig.add_trace(go.Scatter(
        x=base_df.index,
        y=scenarios_ols.loc[scenarios_ols["Scenario"]==scen, "pred"],
        name=scen, line=dict(color=color, dash=dash, width=2)
    ))

for scen, color in zip(
    ["Observé (SARIMAX)", "Sans vaccination (SARIMAX)", "Sans MNP (SARIMAX)", "MNP maintenus +50% (SARIMAX)"],
    ["black", "orange", "red", "green"]
):
    dash = "longdash" if scen != "Observé (SARIMAX)" else "dot"
    fig.add_trace(go.Scatter(
        x=base_df.index,
        y=scenarios_sx.loc[scenarios_sx["Scenario"]==scen, "pred"],
        name=scen, line=dict(color=color, dash=dash, width=1.5)
    ))

fig.add_shape(type="line", x0=COVID_START, x1=COVID_START, y0=0, y1=1, xref="x", yref="paper",
              line=dict(color="red", width=2, dash="dash"))
fig.add_shape(type="line", x0=VACC_START, x1=VACC_START, y0=0, y1=1, xref="x", yref="paper",
              line=dict(color="green", width=2, dash="dash"))
fig.update_layout(
    title="Comparaison des scénarios contrefactuels — OLS vs SARIMAX",
    xaxis_title="Semaine ISO",
    yaxis_title="RSV simulé (taux hebdomadaire)",
    height=700,
    legend=dict(orientation="h", y=-0.25),
    template="plotly_white"
)
fig.show()

In [ ]:
# ==========================================
# 🧠 Interprétation automatique — Comparaison OLS vs SARIMAX
# ==========================================
from IPython.display import Markdown, display

def interpret_model_comparison(summary_ols, summary_sx):
    # On extrait les deltas sous forme de dictionnaires
    ols_vals = {row["Scénario"]: row["Δ_cumulé_%"] for _, row in summary_ols.iterrows()}
    sx_vals  = {row["Scénario"]: row["Δ_cumulé_%"] for _, row in summary_sx.iterrows()}

    text = "## 🧠 Interprétation comparative OLS vs SARIMAX\n\n"
    text += "Cette section compare la **réaction des deux modèles** (OLS et SARIMAX) face aux mêmes scénarios contrefactuels.\n\n"

    # --- Cas 1 : Sans vaccination
    if "Sans vaccination (OLS)" in ols_vals and "Sans vaccination (SARIMAX)" in sx_vals:
        diff_vacc = ols_vals["Sans vaccination (OLS)"] - sx_vals["Sans vaccination (SARIMAX)"]
        if diff_vacc > 0:
            text += f"🟠 **Sans vaccination** → L’OLS estime un impact plus fort (+{diff_vacc:.1f} points) que le SARIMAX, suggérant qu’il amplifie la dépendance à la couverture vaccinale.\n"
        else:
            text += f"🟢 **Sans vaccination** → Le SARIMAX réagit plus fortement ({abs(diff_vacc):.1f} points de plus), indiquant une sensibilité temporelle accrue aux variations de vaccination.\n"

    # --- Cas 2 : Sans MNP
    if "Sans MNP (OLS)" in ols_vals and "Sans MNP (SARIMAX)" in sx_vals:
        diff_mnp = ols_vals["Sans MNP (OLS)"] - sx_vals["Sans MNP (SARIMAX)"]
        if diff_mnp > 0:
            text += f"🔴 **Sans mesures non pharmaceutiques (MNP)** → L’OLS prévoit une remontée plus brutale du RSV (+{diff_mnp:.1f} points), traduisant un effet direct et immédiat.\n"
        else:
            text += f"🟣 **Sans MNP** → Le SARIMAX projette une hausse plus durable ({abs(diff_mnp):.1f} points), reflétant la mémoire temporelle du signal RSV.\n"

    # --- Cas 3 : MNP maintenus +50%
    if "MNP maintenus +50% (OLS)" in ols_vals and "MNP maintenus +50% (SARIMAX)" in sx_vals:
        diff_high = ols_vals["MNP maintenus +50% (OLS)"] - sx_vals["MNP maintenus +50% (SARIMAX)"]
        if diff_high < 0:
            text += f"🟢 **MNP maintenus (+50%)** → L’OLS anticipe une baisse plus forte ({abs(diff_high):.1f} points de plus) → il traduit un effet immédiat des gestes barrières.\n"
        else:
            text += f"🟠 **MNP maintenus (+50%)** → Le SARIMAX amplifie la baisse à long terme (+{diff_high:.1f} points), traduisant un effet cumulatif dans le temps.\n"

    text += "\n---\n\n"
    text += "### 🧩 En résumé :\n"
    text += "- **OLS** → modèle *explicatif instantané*, il réagit fortement aux variations directes des variables exogènes (vaccin, MNP, température). Il capture bien les ruptures mais sans mémoire temporelle.\n"
    text += "- **SARIMAX** → modèle *prédictif à mémoire*, plus stable dans le temps. Il amortit les chocs immédiats mais prolonge leurs effets sur plusieurs semaines.\n\n"
    text += "➡️ Ensemble, ils offrent **deux visions complémentaires** :\n"
    text += "- L’OLS aide à comprendre *le pourquoi des variations* (effets directs).\n"
    text += "- Le SARIMAX aide à projeter *le comment dans le temps* (effets dynamiques)."

    display(Markdown(text))

# Appel automatique
interpret_model_comparison(summary_ols, summary_sx)
